- O que é o DES? (overview)
- Comparar com a cifra de Feistel
- Falar sobre confusão e difusão
- Explicar como ele funciona (black box)

In [265]:
from typing import List

Tabelas de permutação, expansão, etc utilizadas

In [266]:
initial_perm = [
    [2, 6, 3, 1, 4, 8, 5, 7],
]

final_perm = [
    [4, 1, 3, 5, 7, 2, 8, 6],
]

keyp = [
    [3, 5, 2, 7, 4, 10, 1, 9, 8, 6],
]

key_comp = [
    [6, 3, 7, 4, 8, 5, 10, 9],
]

dbox = [
    [4, 1, 2, 3, 2, 3, 4, 1],
]

pbox = [
    [2, 4, 3, 1],
]

shift_table = [1, 2]

sbox = [
    [
        [1, 0, 3, 2],
        [3, 2, 1, 0],
        [0, 2, 1, 3],
        [3, 1, 3, 2],
    ],
    [
        [0, 1, 2, 3],
        [2, 0, 1, 3],
        [3, 0, 1, 0],
        [2, 1, 0, 3],
    ],
]

Funções auxiliares para trabalhar com valores binários
Criei a classe bin porque o python ignora zeros a esquerda e o algoritmo precisa manter o tamanho dos
dados binários consistentes

In [267]:
class Bin:

    def __init__(self, value: int, size: int):
        self.value = value
        self.size = size

    def __repr__(self):
        return f"Bin(value={self.to_hex()}, size={self.size})"

    def __str__(self):
        return self.to_hex()

    def __eq__(self, other):
        if isinstance(other, Bin):
            return self.value == other.value and self.size == other.size
        return False

    def to_bin(self):
        return f"0b{self.value:0{self.size}b}"

    def to_hex(self):
        hex_digits = (self.size + 3) // 4
        return f"0x{self.value:0{hex_digits}X}"


def extract_bits(bits: Bin, table: List[List[int]]) -> Bin:
    extracted_bits = []
    for row in table:
        for bit_pos in row:
            bit = (bits.value >> (bits.size - bit_pos)) & 1
            extracted_bits.append(bit)

    result = 0
    for bit in extracted_bits:
        result = (result << 1) | bit

    return Bin(result, len(extracted_bits))


def halve(bits: Bin):
    half = (bits.size + 1) // 2
    left = (bits.value >> half) & ((1 << half) - 1)
    right = bits.value & ((1 << half) - 1)

    return Bin(left, half), Bin(right, half)


def split_bits(bits: Bin, chunk_size: int):
    chunks: List[Bin] = []
    for i in range(0, bits.size, chunk_size):
        chunk_value = (bits.value >> (bits.size - i - chunk_size)) & (
            (1 << chunk_size) - 1
        )
        chunks.append(Bin(chunk_value, chunk_size))
    return chunks


def fuse_bits(*bins: Bin):
    total_value = 0
    total_size = 0
    for b in bins:
        total_value = (total_value << b.size) | b.value
        total_size += b.size
    return Bin(total_value, total_size)


def swap_halves(bits: Bin):
    left, right = halve(bits)
    return fuse_bits(right, left)


def circular_left_shift(bits: Bin, shift_value: int):
    shift_value %= bits.size
    discarded = bits.value >> (bits.size - shift_value)
    mask = (1 << bits.size) - 1
    shifted = bits.value << shift_value & mask
    return Bin(shifted | discarded, bits.size)


def xor(bits1: Bin, bits2: Bin):
    return Bin(bits1.value ^ bits2.value, bits1.size)

In [268]:
def generate_subkeys(key: Bin):
    pc1 = permuted_choice_1(key)
    left, right = halve(pc1)

    subkeys: List[Bin] = []
    for i in range(2):
        left = circular_left_shift(left, shift_table[i])
        right = circular_left_shift(right, shift_table[i])
        round_key = permuted_choice_2(left, right)
        subkeys.append(round_key)

    return subkeys


def permuted_choice_1(key: Bin):
    return extract_bits(key, keyp)


def permuted_choice_2(left: Bin, right: Bin):
    bits = fuse_bits(left, right)
    return extract_bits(bits, key_comp)

In [269]:
def feistel_encrypt(plaintext: Bin, key: Bin, test=False,reverse=False):
    #print("\n")
    #print(f"Initial Permutation: {plaintext.to_bin()}") #p1
    subkeys = generate_subkeys(key)
    if reverse:
        subkeys.reverse()

    left, right = halve(plaintext)
    #print(left.to_bin(),right.to_bin())
    round_data: List[List[Bin]] = []

    for i in range(2):
        new_left = right

        sk = subkeys[i]
        magled_right = mangler(right, sk)
        new_right = xor(magled_right, left)

        left = new_left
        right = new_right
        #print(f"left: {left.to_bin()}, right:{right.to_bin()}\n")
        round_data.append([left, right, sk])

    result = fuse_bits(left, right)

    # TODO: add logger so I don't have this round_data as return
    return (result, round_data) if test else result


def mangler(bits: Bin, round_subkey: Bin):
    expanded_right = dbox_expansion(bits)
    #print(f"Expanded: {expanded_right.to_bin()}")
    xored_right = xor(expanded_right, round_subkey)
    #print(f"Xored: {xored_right.to_bin()}")
    chunks = split_bits(xored_right, 4)
    #print(f"Chunks: {chunks}")
    fused = sbox_substitution(chunks)
    #print(f"Sbox result: {fused.to_bin()}")
    result = pbox_permutation(fused)
    #print(f"Pbox result: {result.to_bin()}")
    return result



def dbox_expansion(bits: Bin):
    return extract_bits(bits, dbox)


def sbox_substitution(chunks: List[Bin]):
    result = []
    for round, chunk in enumerate(chunks):
        line = extract_bits(chunk, [[1, 4]])
        column = extract_bits(chunk, [[2, 3]])
        sbox_value = sbox[round][line.value][column.value]
        result.append(Bin(sbox_value, 2))

    return fuse_bits(*result)


def pbox_permutation(bits: Bin):
    return extract_bits(bits, pbox)

In [270]:
def des_encryption(plaintext: Bin, key: Bin):
    ciphertext1 = initial_permutation(plaintext)
    ciphertext2 = feistel_encrypt(ciphertext1, key)
    ciphertext3 = swap_halves(ciphertext2)
    ciphertext4 = final_permutation(ciphertext3)
    return ciphertext4


def des_decryption(plaintext: Bin, key: Bin):
    ciphertext1 = initial_permutation(plaintext)
    ciphertext2 = feistel_encrypt(ciphertext1, key, reverse=True)
    ciphertext3 = swap_halves(ciphertext2)
    ciphertext4 = final_permutation(ciphertext3)
    return ciphertext4


def initial_permutation(plaintext: Bin):
    return extract_bits(plaintext, initial_perm)


def final_permutation(ciphertext: Bin):
    return extract_bits(ciphertext, final_perm)

In [271]:
def test_subkey_generation(example_key):
    try:
        expected_keys = [
            Bin(0b10100100, 8),
            Bin(0b01000011, 8),
        ]

        generated_keys = generate_subkeys(example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING SUBKEYS GENERATION".center(40))
    print("-" * 40)
    print("EXPECTED KEY   | GENERATED KEY")
    for g1, g2 in zip(expected_keys, generated_keys):
        print(f"{g1.to_bin():<14} | {g2.to_bin()}")

    all_match = all(g1 == g2 for g1, g2 in zip(expected_keys, generated_keys))
    if all_match:
        print("\nTest passed: All subkeys match expected values.")
    else:
        print("\nTest failed: Subkeys do not match expected values.")


def test_feistel_encrypt(example_plaintext, example_key):
    try:
        expected_rounds = [
            [Bin(0b1101, 4), Bin(0b1010, 4), Bin(0b10100100, 8)],
            [Bin(0b1010, 4), Bin(0b0010, 4), Bin(0b01000011, 8)],
        ]

        initial_permutation = extract_bits(example_plaintext, initial_perm)
        example_key = Bin(0b1010000010, 10)
        rounds_data = feistel_encrypt(initial_permutation, example_key, test=True)[1]

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING FEISTEL ENCRYPTION".center(40))
    print("-" * 40)

    print("EXPECTED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in expected_rounds:
        print(f"{l.to_bin():<10} | {r.to_bin():<10} | {sk.to_bin()}")

    print("\nGENERATED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in rounds_data:
        print(f"{l.to_bin():<10} | {r.to_bin():<10} | {sk.to_bin()}")

    print()
    all_match = all(g1 == g2 for g1, g2 in zip(expected_rounds, rounds_data))
    if all_match:
        print("Test passed: All rounds match expected values.")
    else:
        print("Test failed: Rounds do not match expected values.")


def test_des_encryption(
    example_plaintext: Bin, example_key: Bin, expected_ciphertext: Bin
):
    try:
        generated_ciphertext = des_encryption(example_plaintext, example_key)
    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES ENCRYPTION".center(40))
    print("-" * 40)

    print(f"{'Plaintext:':<22}{example_plaintext.to_bin()}")
    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Expected Ciphertext:':<22}{expected_ciphertext.to_bin()}")
    print(f"{'Obtained Ciphertext:':<22}{generated_ciphertext.to_bin()}\n")

    assert (
        generated_ciphertext == expected_ciphertext
    ), "Test failed: Ciphertext does not match expected value."

    print("Test passed: Ciphertext matches expected value.")


def test_des_decryption(
    example_ciphertext: Bin, example_key: Bin, expected_plaintext: Bin
):
    try:
        generated_plaintext = des_decryption(example_ciphertext, example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES DECRYPTION".center(40))
    print("-" * 40)

    print(f"{'Ciphertext:':<22}{example_ciphertext.to_bin()}")
    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Expected Plaintext:':<22}{expected_plaintext.to_bin()}")
    print(f"{'Obtained Plaintext:':<22}{generated_plaintext.to_bin()}\n")

    assert (
        generated_plaintext == expected_plaintext
    ), "Test failed: Ciphertext does not match expected value."

    print("Test passed: Ciphertext matches expected value.")

In [272]:
# example_plaintext1 = Bin(0b11010111, 8)
# example_key1 = Bin(0b1010000010, 10)
# expected_ciphertext1 = Bin(0xC0B7A8D05F3A829C, 8)

example_plaintext2 = Bin(0b10010111 , 8)
example_key2 = Bin(0b1010000010, 10)
expected_ciphertext2 = Bin(0b00111000 , 8)

test_subkey_generation(example_key2)
test_feistel_encrypt(example_plaintext2, example_key2)
test_des_encryption(example_plaintext2, example_key2, expected_ciphertext2)
test_des_decryption(expected_ciphertext2, example_key2, example_plaintext2)

      

TESTING SUBKEYS GENERATION      
----------------------------------------
EXPECTED KEY   | GENERATED KEY
0b10100100     | 0b10100100
0b01000011     | 0b01000011

Test passed: All subkeys match expected values.
      

TESTING FEISTEL ENCRYPTION      
----------------------------------------
EXPECTED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0b1101     | 0b1010     | 0b10100100
0b1010     | 0b0010     | 0b01000011

GENERATED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0b1101     | 0b1010     | 0b10100100
0b1010     | 0b0010     | 0b01000011

Test passed: All rounds match expected values.
        

TESTING DES ENCRYPTION        
----------------------------------------
Plaintext:            0b10010111
Key:                  0b1010000010
Expected Ciphertext:  0b00111000
Obtained Ciphertext:  0b00111000

Test passed: Ciphertext matches expected value.
        

TESTING DES DECRYPTION        
----------------------------------------
Ciphertext:           0b00111000
Key:        

Referências

https://www.geeksforgeeks.org/data-encryption-standard-des-set-1/

https://www.geeksforgeeks.org/shift-micro-operations-in-computer-architecture/

https://aprender3.unb.br/pluginfile.php/3076851/mod_resource/content/1/Aula%2008%20-%20Cifra%20de%20Bloco%20-%20DES%20%282%29.pdf